In [12]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing as mp
from sklearn.utils import class_weight
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter

In [13]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [16]:
df = pd.read_csv('data/news_predictions/news_2015_predictions.csv')

In [17]:
df = df.drop(['Unnamed: 0'], axis=1)
df  = df.dropna(subset=['transcript'])
df = df.reset_index(drop=True)

In [18]:
df_above_threshold = df.loc[df.top1_acc >= 40]
print(len(df_above_threshold))

582219


In [19]:
df_above_threshold_and_close = df_above_threshold.loc[(df.top1_acc <= (df.top2_acc + 10))]
print(len(df_above_threshold_and_close))

11074


In [20]:
df_close = df.loc[(df.top1_acc <= (df.top2_acc + 10))]
print(len(df_close))

617370


In [9]:
df_above_threshold_and_not_close.parliament.value_counts()

1    570759
0       386
Name: parliament, dtype: int64

In [24]:
rows = [['total', 1608102, 1], ['above_40', 582219, 582219 / 1608102], ['close_10', 617370, 617370 / 1608102], ['above_40_and_close_10', 11074, 11074 / 1608102]]
df = pd.DataFrame(rows, columns=['type', 'count', 'fraction'])
df.to_csv('articles_threshold_count_2015.csv')

In [25]:
df

,type,count,fraction
0,total,1608102,1.000000
1,above_40,582219,0.362054
2,close_10,617370,0.383912
3,above_40_and_close_10,11074,0.006886


In [5]:
df_close = df.loc[df.top1_acc <= (df.top2_acc + 10)]

In [7]:
len(df_close) / len(df)

0.38119659808416867

In [13]:
df_parliament = df.loc[(df.top1_topic == 'Parliament, government and politics')]

In [9]:
df_close_parliament = df.loc[(df.top1_topic == 'Parliament, government and politics') & (df.top1_acc <= (df.top2_acc + 10))]

In [14]:
len(df_close_parliament) / len(df_parliament)

0.40721140062399863

In [17]:
df_close_parliament.top2_topic.value_counts().to_csv('parliament_second_close_topics.csv')